In [21]:
from keras.models import model_from_json
import numpy as np
from astropy.io.fits import getdata, PrimaryHDU

In [5]:
def im2patch(I,sz,szp):
    delt,n1,n2=im2patch_len(I.shape,sz,szp)
    idx=0
    patches=np.zeros((n1*n2,1,sz,sz))
    for i in range(n1):
        for j in range(n2):
            patches[idx,:,:,:]=I[i*szp:i*szp+sz,j*szp:j*szp+sz]
            idx+=1
    return patches

In [6]:
def im2patch_len(imzize,sz,szp):
    delt=int(np.ceil((sz-szp)*1.0/2))
    n1=int(np.floor((imzize[0]-2*delt)*1.0/szp))
    n2=int(np.floor((imzize[1]-2*delt)*1.0/szp))
    return delt,n1,n2

In [7]:
def patch2im(patches,imsize,sz,szp):
    delt,n1,n2=im2patch_len(imsize,sz,szp)
    I=np.zeros(imsize)
    idx=0
    for i in range(n1):
        for j in range(n2):
            I[i*szp+delt:(i+1)*szp+delt,j*szp+delt:(j+1)*szp+delt]=patches[idx,:,:,:]
            idx+=1
    return I 

In [3]:
model = model_from_json(open('PaperPretrained/32x32_10x10-64_6x6-16_5x5-1_PSF64_sigma0.010_M31.json').read())
model.load_weights('PaperPretrained/32x32_10x10-64_6x6-16_5x5-1_PSF64_sigma0.010_M31.h5')

/usr/local/lib64/python3.6/site-packages/keras/engine/base_layer.py:1124: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(name="convolution2d_4", activity_regularizer=None, trainable=True, input_dtype="float32", batch_input_shape=[None, 1, ..., activation="relu", kernel_size=(10, 10), filters=64, strides=[1, 1], padding="valid", data_format="channels_first", kernel_initializer="glorot_uniform", kernel_regularizer=None, bias_regularizer=None, kernel_constraint=None, bias_constraint=None, use_bias=True)`
  return cls(**config)
/usr/local/lib64/python3.6/site-packages/keras/engine/base_layer.py:1124: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(name="convolution2d_5", activity_regularizer=None, trainable=True, activation="relu", kernel_size=(6, 6), filters=16, strides=[1, 1], padding="valid", data_format="channels_first", kernel_initializer="glorot_uniform", kernel_regularizer=None, bias_regularizer=None, kernel_constraint=None, bias_constraint=Non

In [12]:
I0_all = [getdata('BadM13I20120611.fits') * 1.0]
I_all=[np.maximum(0,I/I.max()) for I in I0_all]

In [13]:
szi = 32
szo = 14

In [14]:
Ipatch = im2patch(I_all[0], szi, szo)

In [15]:
Ipred = model.predict(Ipatch)

In [16]:
Irec = patch2im(Ipred, I_all[0].shape, szi, szo)

In [17]:
Irec.shape

(2048, 2048)

In [22]:
hdu = PrimaryHDU(Irec)
hdu.writeto('PaperPretrained/paper_prediction.h5')